# TFLite Converter with TensorFlow 2.x 
![simple_nn](media/miscellaneous/tf_logo.png "TF Logo")

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [19]:
PATH_DIR = Path.cwd()
dataset_dir = PATH_DIR.joinpath('bin/tf_tutorial_2')
saved_model_dir = dataset_dir.joinpath('original_model')
saved_h5_dir = dataset_dir.joinpath('model_original.h5')

![simple_nn](media/tf_tutorial_3/tensorflow_lite_framework.png "TF-Lite Landscape")

# 1.0 Train a simple CNN on MNIST

In [4]:
# import the datatset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [5]:
# normalize dataset
def normalize(x, y):
    return x / 255, y

In [6]:
# prepare the data
ds_train = ds_train.map(normalize).cache().batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(normalize).batch(32)

In [7]:
# create a simple cnn model
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28,28,1)),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),   
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10, activation='softmax' )
])

In [8]:
# compile
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'])

In [9]:
# train
history = model.fit(ds_train, steps_per_epoch=len(X_train)/32, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1151 - accuracy: 0.9657
Epoch 2/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0406 - accuracy: 0.9881
Epoch 3/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0295 - accuracy: 0.9913
Epoch 4/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0212 - accuracy: 0.9934
Epoch 5/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0150 - accuracy: 0.9952
Epoch 6/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0157 - accuracy: 0.9948
Epoch 7/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0105 - accuracy: 0.9965
Epoch 8/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0091 - accuracy: 0.9972
Epoch 9/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0091 - accuracy: 0.9967
Epoch 10/15
1875/1875 [==============================] - 11s 6ms/step - lo

In [10]:
# evaluate
model.evaluate((ds_test))

313/313 [==============================] - 0s 2ms/step - loss: 0.0451 - accuracy: 0.9909


[0.04507521912455559, 0.9908999800682068]

In [20]:
# save model
model.save(saved_model_dir)
model.save(saved_h5_dir)

INFO:tensorflow:Assets written to: /home/vitto/Documents/Random/EscVMYT/Notebooks/bin/tf_tutorial_2/original_model/assets


INFO:tensorflow:Assets written to: /home/vitto/Documents/Random/EscVMYT/Notebooks/bin/tf_tutorial_2/original_model/assets


## 2.0 TF-Lite simple conversion

In [ ]:
# from keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# or from tf saved model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# last from concrete functions
converter = tf.lite.TFLiteConverter.from_concrete_funcions(tf_path_concrete_functions)

In [21]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())

In [22]:
# start conversion
tflite_model = converter.convert()

In [23]:
# save model
tflite_model_file = dataset_dir.joinpath('model_fp32.tflite')
tflite_model_file.write_bytes(tflite_model)

1164772

# 3.0 Float16 quantization

In [31]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())

In [32]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [33]:
tflite_model = converter.convert()

In [34]:
# save model
tflite_model_file = dataset_dir.joinpath('model_fp16.tflite')
tflite_model_file.write_bytes(tflite_model)

592352

# 4.0 Dynamic int8 range quantization

In [25]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())

In [26]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [29]:
tflite_model = converter.convert()

In [30]:
# save model
tflite_model_file = dataset_dir.joinpath('model_int8_dynamic.tflite')
tflite_model_file.write_bytes(tflite_model)

304496

# 5.0 Integer quantization with float fallback

In [48]:
num_calibration_steps = 1 # at least 100

def representative_dataset_gen():
      for i in range(num_calibration_steps):
        # Remember to pre-process your dataset as your training
        imgs = X_train[i:i+1]
        imgs = imgs / 255
        yield [imgs.astype('float32')]

In [36]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())

In [49]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen

In [50]:
tflite_model = converter.convert()

In [51]:
# save model
tflite_model_file = dataset_dir.joinpath('model_int8_fb.tflite')
tflite_model_file.write_bytes(tflite_model)

318400

# 6.0 Full integer quantization (integer only)

## 6.1 With TF >= 2.3

In [52]:
num_calibration_steps = 1 # at least 100

def representative_dataset_gen():
      for i in range(num_calibration_steps):
        # Remember to pre-process your dataset as your training
        imgs = X_train[i:i+1]
        imgs = imgs / 255
        yield [imgs.astype('float32')]

In [54]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())

In [55]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8

In [56]:
tflite_model = converter.convert()

In [57]:
# save model
tflite_model_file = dataset_dir.joinpath('model_int8.tflite')
tflite_model_file.write_bytes(tflite_model)

318144

## 6.2 With TF < 2.3

In [58]:
num_calibration_steps = 1 # at least 100

def representative_dataset_gen():
      for i in range(num_calibration_steps):
        # Remember to pre-process your dataset as your training
        imgs = X_train[i:i+1]
        imgs = imgs / 255
        yield [imgs.astype('float32')]

In [60]:
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(dataset_dir.joinpath('model_original.h5'))

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [61]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_dataset_gen
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [62]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpcgg0kx2e/assets


INFO:tensorflow:Assets written to: /tmp/tmpcgg0kx2e/assets


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


INFO:tensorflow:Restoring parameters from /tmp/tmpcgg0kx2e/variables/variables


...

INFO:tensorflow:Restoring parameters from /tmp/tmpcgg0kx2e/variables/variables


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [63]:
# save model
tflite_model_file = dataset_dir.joinpath('model_int8.tflite')
tflite_model_file.write_bytes(tflite_model)

315440